<a href="https://colab.research.google.com/github/chinmaychahar/STEM-AWAY/blob/main/STEM_AWAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
import time
from datetime import datetime
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import json
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
from selenium import webdriver
#PROXY = "117.197.119.151:8080"
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#options.add_argument('--proxy-server=%s' % PROXY)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome(options=options)
#wait = WebDriverWait(driver, 10)
    # retrive url in headless browser
driver.get('https://discuss.codecademy.com/c/get-help/1801')

#print(wd.page_source)  # results

In [ ]:
topicHTML = driver.page_source
Topic = BeautifulSoup(topicHTML, 'html.parser')

In [ ]:
def get_title(topicSoup):
        topicName = topicSoup.find('a', class_='raw-topic-link')
        if topicName == None:
            return str(0)
        else:
            topicName = topicName.text
        # Remove leading and trailing spaces and newlines
        topicName = topicName.replace('\n', '').strip()
        return topicName


In [ ]:
get_title(Topic)

'About the Get Help category'

In [ ]:
def get_category_and_tags(topicSoup):
        topicCategoryDiv = topicSoup.find(
            'div', class_='link-bottom-line')
        categoryAnchors = topicCategoryDiv.find('span', class_='category-name')
        tagAnchors = topicCategoryDiv.find_all(
            'a', class_='discourse-tag bullet')

        category = categoryAnchors.text
        tagList = []
        for anchor in tagAnchors:
            tagList.append(anchor.text)
        return category

In [ ]:
get_category_and_tags(Topic)

'Get Help'

In [ ]:
def get_author_and_commenters( topicSoup):
        names_tag = topicSoup.find("td", class_="posters")
        names = names_tag.find_all('a')
        authorList = []
        for name in names:
          #print(name['data-user-card'])
          authorList.append(name['data-user-card'])
        
        authorList = list(set(authorList))
        if (len(authorList) == 1):
            author = authorList[0]
            commenters = []
            return author, commenters
        else:
            author = authorList[0]
            commenters = authorList[1:]
            return author, commenters

get_author_and_commenters(Topic)

('alexc', ['oduffy'])

Trying next

In [ ]:
from selenium import webdriver
#PROXY = "117.197.119.151:8080"
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#options.add_argument('--proxy-server=%s' % PROXY)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome(options=options)
#wait = WebDriverWait(driver, 10)
    # retrive url in headless browser
driver.get('https://discuss.codecademy.com/c/get-help/1801')

#print(wd.page_source)  # results

In [ ]:
for i in range(3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

data = driver.find_elements_by_css_selector('a[class = "title raw-link raw-topic-link"]')
titles = [x.text for x in data]
links = [x.get_attribute('href') for x in data]

subcat_data = driver.find_elements_by_css_selector('span[class = "category-name"]')
subcategory = [x.text for x in subcat_data[1:]]

contents = []
for link in links:
    driver.get(link)
    data = driver.find_elements_by_css_selector('div[class = "cooked"]')
    content = [x.text.replace('\n', ' ') for x in data]
    contents.append(content)

time.sleep(5)
driver.quit()

In [ ]:
# Taking Posts Content
posts = []
responses = []
for line in contents:
    #print(line)
    posts.append(line[0])
    responses.append(line[1:2])


In [ ]:
def response_df(urls, resps):
    column_names = ['URL', 'Response Text']
    df_resps = pd.DataFrame(columns = column_names)
    for url,line in zip(urls, resps):
        n = len(line)
        sub_urls = [url] * n
        new_df = pd.DataFrame(list(zip(sub_urls, line)), columns = column_names)
        df_resps = pd.concat([df_resps, new_df])
    return df_resps

In [ ]:

DF1 = pd.DataFrame(titles, columns=['Title'])
DF1['Category'] = subcategory
#DF1['Sub-category'] = subcategory
DF1['Post Text'] = posts
DF1['URL'] = links
DF1['Leading Comments'] = responses

In [ ]:
DF1.head()

,Title,Category,Post Text,URL,Leading Comments
0,About the Get Help category,Get Help,"Ask questions, get help with an exercise, and ...",https://discuss.codecademy.com/t/about-the-get...,[]
1,Can’t fix this typo,HTML,"Hi all, I just have a quick question. Just loo...",https://discuss.codecademy.com/t/cant-fix-this...,"[Hey, and welcome to the forums! I think it mi..."
2,Jamming project next steps,JavaScript,"Hi, this is my version of Jamming project: htt...",https://discuss.codecademy.com/t/jamming-proje...,[]
3,Importing a Method From a Specific Class In a ...,Python,Sometimes l want to import methods from differ...,https://discuss.codecademy.com/t/importing-a-m...,[You need to carefully check what you’re impor...
4,Introduction To Machine Learning in R: Predict...,R category,"Hey all, Getting to the end of the course on R...",https://discuss.codecademy.com/t/introduction-...,[I just finished with this activity and ran in...


In [ ]:
DF2 = response_df(links, responses)

In [ ]:
DF1.to_csv("DF1.csv", index = False)
DF2.to_csv("DF2.csv", index = False)

EDA

In [ ]:
DF1.isna().sum()

Title               0
Category            0
Post Text           0
URL                 0
Leading Comments    0
dtype: int64

In [ ]:
DF1.dropna(axis=0, inplace=True)

In [ ]:
DF1.Category.unique()

array(['Get Help', 'HTML', 'JavaScript', 'Python', 'R category',
       'React.js', 'SQL', 'General Programming', 'Java', 'jQuery', 'CSS',
       'Node.js', 'C++', 'Codecademy', 'Git', 'Express.js'], dtype=object)

In [ ]:
DF1.columns

Index(['Title', 'Category', 'Post Text', 'URL', 'Leading Comments'], dtype='object')

In [ ]:
DF1.drop(columns=['URL'], inplace=True)

In [ ]:
DF1 = DF1.iloc[1: , :]
#remove first row as it was system generated FAQ with no comments

In [ ]:
DF1.head()

,Title,Category,Post Text,Leading Comments
1,Can’t fix this typo,HTML,"Hi all, I just have a quick question. Just loo...","[Hey, and welcome to the forums! I think it mi..."
2,Jamming project next steps,JavaScript,"Hi, this is my version of Jamming project: htt...",[]
3,Importing a Method From a Specific Class In a ...,Python,Sometimes l want to import methods from differ...,[You need to carefully check what you’re impor...
4,Introduction To Machine Learning in R: Predict...,R category,"Hey all, Getting to the end of the course on R...",[I just finished with this activity and ran in...
5,Meal Maker project - Cannot read property ‘len...,JavaScript,"const menu = { _courses: { appetizers: [], mai...",[Veery small typo: the property in _courses is...


In [ ]:
DF1.rename(columns={'Leading Comments': 'texts', 'Category': 'labels'}, inplace=True)

In [ ]:
DF1.head()

,Title,labels,Post Text,texts
1,Can’t fix this typo,HTML,"Hi all, I just have a quick question. Just loo...","[Hey, and welcome to the forums! I think it mi..."
2,Jamming project next steps,JavaScript,"Hi, this is my version of Jamming project: htt...",[]
3,Importing a Method From a Specific Class In a ...,Python,Sometimes l want to import methods from differ...,[You need to carefully check what you’re impor...
4,Introduction To Machine Learning in R: Predict...,R category,"Hey all, Getting to the end of the course on R...",[I just finished with this activity and ran in...
5,Meal Maker project - Cannot read property ‘len...,JavaScript,"const menu = { _courses: { appetizers: [], mai...",[Veery small typo: the property in _courses is...


In [ ]:
DF1.texts = DF1.texts.apply(lambda y: np.nan if len(y)==0 else y)

In [ ]:
DF1.labels.unique()

array(['HTML', 'JavaScript', 'Python', 'R category', 'React.js',
       'Get Help', 'SQL', 'General Programming', 'Java', 'jQuery', 'CSS',
       'Node.js', 'C++', 'Codecademy', 'Git', 'Express.js'], dtype=object)

DONE WITH EDA

MODEL NOW

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

X = DF1.texts
y = DF1.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 42)

In [ ]:
curr = X_train
X_train = []
for c in curr:
  print(c)

print(X_train)

['Hi @codejumper03379 what does your topicSlice look like? You dispatch objects as payload. I’m not sure if a shorthand works here. But seeing what the topic state and reducer that receive the payload would be relevant here.']
nan
['Solved by tgrtim in post #4 You have two different but similarly named attributes num_of_children and new_num_children. What does your estimated cost function make use of?']
nan
['https://www.codecademy.com/paths/data-science/tracks/dscp-python-fundamentals/modules/dscp-python-strings/projects/ds-python-strings-project 34 Here is the link']
['Hey, and welcome to the forums! I think it might also be helpful if you could attach a screenshot of the code you’re trying to fix? :))']
['if you change the gpa on line 6 to a lower score: gpa = 1.5 The condition still evaluates to true, showing you that you have the requirements to graduate, even though you don’t. you have a problem with your right conditional statement']
['El error que te da es éste: AttributeError:

In [ ]:
logreg = Pipeline([('vect', TfidfVectorizer(stop_words=None)),
                   ('tfidf', TfidfTransformer()),
                   ('clf', LogisticRegression(n_jo  bs=1, C=10.526405263157894, max_iter=200)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
res31 = accuracy_score(y_pred, y_test)
print(classification_report(y_test, y_pred))

ValueError: ignored

MODEL -02

In [ ]:
X = DF1.texts
y = DF1.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
X_train.head()

45     [I also need some help here. I’m in the same s...
13     [Solved by taylorbalbi in post #2 Found the an...
16     [What do you mean, exactly? The p tag in your ...
71     [@ data6260682876 If u don’t use selectors wer...
109    [El error que te da es éste: AttributeError: m...
Name: texts, dtype: object

In [ ]:
y_train.head()

45     Python
13     Python
16       HTML
71        CSS
109    Python
Name: labels, dtype: object

In [ ]:
from sklearn.linear_model import LogisticRegression

def Logistic_Regression(X_train, X_test, y_train, y_test):
    logreg = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                   ])
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict (X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,target_names=my_categories))
    return accuracy_score(y_pred, y_test)
res1331 = Logistic_Regression(X_train, X_test, y_train, y_test)

AttributeError: ignored